# 2.1 Прикрой свой з** инструкциями assert

- `assert` – средство отладки, которое проверяет условие; если оно `True`, то ничего не происходит и программа продолжает выполнение, но если же условие возвращает `False`, то вызывается исключение `AssertionError` с необязательным сообщением об ошибке.

## Пример:

In [9]:
def apply_discount(product: dict, discount: float) -> int:
    """
    Applies discount to current product price.
    
    :return: price of product with apllied discount.
    """
    price = int(product["price"] * (1 - discount))
    assert 0 <= price <= product["price"]
    
    return price

In [10]:
# To avoid problems with rounding: 149,00$ -> 14900.
shoes = {'name': "Cool shoes", 'price': 14900}

apply_discount(shoes, 0.25)

11175

In [11]:
# Try to apply invalid discount.
# 200%
apply_discount(shoes, 2.)

AssertionError: 

## Почему бы просто не пробросить исключение?

- Дело в том, что инструкция `assert` предназначена для того, чтобы сообщать разработчикам о *неустранимых* ошибках в программе.

- Инструкция `assert` **не предназначена** для того, чтобы сигнализировать об ожидаемых ошибочных условиях (таких как: "File was not found"), где пользователь может предпринять корректирующие действия или просто попробовать ещё раз.

- Инструкции `assert` призваны быть *внутренними самопроверками* (***internal selfchecks***) нашей программы. Они работают путём выявления условий ***возникновение которых в исходном коде невозможно*** и дают понять, что в программе есть ошибка.

***Главный поинт:*** `assert` – *средство отладки*, а не *механизм обработки ошибок в рантайме*! 
**Если в программе нет ошибок, исключение** `AssertionError` **никогда не должно возникнуть.**

## Синтаксис инструкции `assert`

Синтаксис инструкции `assert` в соответствии с [docs.python.org](https://docs.python.org/3/reference/simple_stmts.html#the-assert-statement):

```python
# ["," expression2] is optional
assert_stmt ::=  "assert" expression1 ["," expression2]
```

Во время исполнения интерпретатор Python преобразовывает каждую инструкцию `assert` в нечто подобное:

```python
if __debug__: # is a gloabal variable which is False, if optimization requested.
    if not expression1:
        raise AssertionError(expression2)
```

## Предостережение № 1: не используйте `assert` для проверки данных

Дело в том, что флаги `-O` и `-OO`, а также переменная `PYTHONOPTIMIZE` в CPython способны превратить любую инструкцию `assert` в `nop` (нулевую операцию), так как глобальная переменная `__debug__` станет `False`.

Рассмотрим код:

In [15]:
def delete_product(prod_id: int, user):
    assert user.is_admin, "there is admin here"
    assert store.has_product(prod_id), "unexpected product"
    store.get_product(prod_id).delete()

Что же произойдёт, если `assert` будут отключены:
- проверка полномочий админа несёт опасность: любой пользователь сможет удалять товары;
- проверка наличия продукта пропускается: теперь `get_product` допускается к несуществующим товарам;

Исправим код!

In [17]:
def delete_product(prod_id: int, user):
    if not user.is_admin():
        raise AuthError("Administrator rights are required for product deleting")    
    if not store.has_product(prod_id):
        raise ValueError("Unexpected product id")   
    store.get_product(prod_id).delete()

## Предостережение № 2: не используйте `assert`, если она вообще не может дать сбой

In [20]:
counter = 123
assert (
    counter == 10,
    'This could be count all elements'
)

<>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
C:\Users\sks23\AppData\Local\Temp\ipykernel_12588\22450635.py:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (


Объект-кортеж всегда выполнит каст в `True`, так как он не пуст – получаем соответствующий `SyntaxWarning`.